# Line Tracing —— CV 1&2&3 Summary

## 阶段一：预处理 (Preprocessing)

1. 灰度化 (Grayscale)​​

    - ​​作用​​：将彩色的RGB图像转换为单通道的灰度图像。

    - ​​输出​​：一个二维矩阵，每个像素点只有一个强度值（Intensity），范围是0-255。

2. ​​高斯滤波 (Gaussian Filtering) 

    - ​​作用​​：​​降噪​​。原始图像中可能存在噪点，这些噪点在后续的边缘检测中会被误认为是“边缘”，导致结果不准确。高斯模糊通过卷积运算，将每个像素点的值与其周围像素点的值进行加权平均，从而平滑图像、抑制噪声。

## 阶段二：边缘检测 (Edge Detection)

Definition: An edge is defined as a region in the image where there is a “significant”
change in the pixel intensity values (or having high contrast) along one
direction in the image, and almost no changes in the pixel intensity values
(or low contrast) along its orthogonal direction.

3. ​​计算梯度 (Gradient Calculation)

    - ​​作用​​：找到图像中亮度变化最剧烈的点，这些点就是潜在的边缘。

    - ​​如何实现​​：使用边缘检测算子（如Sobel）与图像进行​​卷积​​。​​这个卷积核的设计目的不再是模糊，而是求差分（近似求导）​​。基于Derivative Theorem:
$$
\frac{d}{dx}(f * g) = f * \frac{d}{dx}g
$$

直到这一步，才真正计算了梯度。前面预处理阶段使用的确实是原始强度值（Intensity）。后续所有步骤都是基于梯度信息，而不是原始强度值。

4. ​​非极大值抑制 (Non-Maximum Suppression, NMS)​​

    - ​​作用​​：​​细化边缘​​。经过梯度计算后，边缘可能是一片粗粗的一条区域。NMS的目的是只保留每个边缘中心点（梯度值最大的点），抑制掉周围非最大的点，从而将“粗边缘”细化成“单像素宽”的精准边缘。

    - 和梯度的联系：​​必须同时使用模和方向​​！NMS的目的是细化边缘，它的做法是：沿着当前像素点的​​梯度方向​​去查看它的两个邻居像素。只有当当前像素的模​​大于​​这两个邻居的模时，它才被保留；否则就被抑制。这样就能确保只留下梯度方向上的极大值，从而得到细化的边缘。如果不知道方向，就无法确定要比较哪两个邻居。

5. ​​滞后阈值与边缘连接 (Hysteresis Thresholding & Edge Linking)​​

    - ​​作用​​：​连线，​确定真正的边缘​​。主要看​梯度的​模​​。用模的大小来判断这是强边缘、弱边缘还是非边缘。通过设置一高一低两个阈值，将梯度值分为三类：

        a. 强边缘（高于高阈值）：肯定是边缘。

        b. 弱边缘（介于高低阈值之间）：可能是边缘，也可能是噪声。

        c. 非边缘（低于低阈值）：肯定不是边缘。

    - ​​边缘连接​​：检查弱边缘像素，如果它与强边缘像素相连，就将其保留为真正的边缘；如果不相连，则认为是噪声并将其抑制。这一步确保了边缘的连续性。

## 阶段三：车道线拟合 (Line Fitting)

经过阶段二，我们得到了一张​​二值图（Binary Image）​​，图中黑色的点就是被检测出的所有边缘点。现在我们需要从这些散乱的点中找出符合直线形状的点集（即车道线）。

6. ​​RANSAC (Random Sample Consensus)​​

    - ​​作用​​：​​鲁棒地拟合模型，排除噪声（异常点）的干扰​​。

    - ​​为什么需要它​​：在边缘二值图中，除了车道线点，肯定还存在很多噪声点。直接用最小二乘法拟合，这些噪声点会严重影响拟合结果，导致直线位置偏离。

    - ​​如何工作​​：RANSAC通过随机抽取少量点来拟合一个模型（如两点确定一条直线），然后计算有多少其他点符合这个模型（即内点 - inliers）。这个过程重复多次，最终选择拥有最多内点的那个模型。它​​能有效地忽略掉那些不符合主流趋势的噪声点​outlier​，得到非常鲁棒和准确的拟合结果。

    - ​​与SVD的关系​​：RANSAC不是替代SVD，而是与它协同工作。​​通常的流程是：用RANSAC从一堆点（可能包含噪声）中找出哪些点是内点（真正的车道线点），然后针对找出的这批“干净的内点”，使用SVD/最小二乘法进行最终精确的直线拟合。    
    ​​
    - 特别说明：RANSAC中的假设可以并行计算，避开loop

    * ​霍夫变换 (Hough Transform)​​ 和 ​​RANSAC​​ 是 ​​竞争/替代​​ 关系。通常会选择其中一种来从边缘点中初步找出候选直线。注：霍夫变换本身通过寻找参数空间的峰值直接给出了模型参数，通常不再需要SVD细化。但RANSAC之后则非常需要。

7. ​最小二乘法 (Least Squares) / SVD​​

    - ​​作用​​：​​从边缘点中拟合出直线方程​​。这是最直观的拟合方法。

    - ​​最小二乘法/SVD​​：如果你已经有一组确信是属于同一条直线的点（比如通过ROI区域选择），可以直接用SVD求解最小二乘解，得到最优的直线拟合。

    - ​​霍夫变换​​：是一种投票机制，非常适合在含有噪声和部分遮挡的图像中检测简单形状（如直线、圆）。

